In [1]:
from nftools import guis
import pylsl
import time
import numpy as np
import mne, re

In [2]:
%load_ext autoreload
%autoreload
%matplotlib qt  
# %gui qt
import warnings; warnings.filterwarnings('ignore') 

### Prepare the EEG Cap

- fix the EEG Cap
- run the `python raw_eo_data --stream`, followed by `/start`
- (re)-start the Cap or USB if it doesn't work, followed by commands above

In [3]:
# make the connection to the EEG Net (EEG Data)
data_stream=pylsl.resolve_byprop("name", "openbci_eeg", timeout=5.0)
if data_stream:
    data_inlet=pylsl.stream_inlet(data_stream[0])
    stream_info = data_inlet.info()
    stream_Fs = stream_info.nominal_srate()
    stream_xml = stream_info.desc()
    chans_xml = stream_xml.child("channels")
    chan_xml_list = []
    ch = chans_xml.child("channel")
    while ch.name() == "channel":
        chan_xml_list.append(ch)
        ch = ch.next_sibling("channel")
    channel_names = [ch_xml.child_value("label") for ch_xml in chan_xml_list]
    data_inlet_dt = data_inlet.time_correction(timeout=5.0)
    sampling_freq = data_stream[0].nominal_srate()
    print('name = %s' % data_stream[0].name())
    print('sampling_freq = %d' % sampling_freq)
    print('channel_count = %d' % data_stream[0].channel_count())
    print('channel_format = %d' % data_stream[0].channel_format())
else:
    raise Exception('No Data Stream Found - Is your EEG Cap running?')

name = openbci_eeg
sampling_freq = 125
channel_count = 16
channel_format = 1


### Get (~ 2-3 minutes) of Eyes Open data

In [4]:
# clear EEG Data buffer
while data_inlet.samples_available(): data_inlet.pull_chunk() 

# collect some data    
np_eo = []
w=guis.AcquireData(sampling_freq, channel_names)
while w.RUNLOOP:
    chunk_data, chunk_times = data_inlet.pull_chunk() # grab from LSL
    if chunk_data:  # if there is any data
        np_eo.append(chunk_data) # add to our list
        w.update(chunk_data) # update the GUI window

# and we make the MNE data file from it        
raw_eo = mne.io.RawArray(np.concatenate(np_eo).T,
                        mne.create_info(channel_names, 
                                    sampling_freq, 
                                    'eeg', 
                                    None)
                       )


### Check Bad Channels and Bad Time Intervals
- Use the MNE Interface to click channels (those are bad)
- Use also the MNE Interface ('a' button) to mark/drag
bad segments

In [5]:
# plot it
if 'w' in locals(): del(w)
raw_eo.plot(scalings='auto');

In [6]:
# Analysis
# extracting the ch names and time information
bad_channels = raw_eo.info['bads']
bad_segments = [(a['onset'], a['duration']) for a in raw_eo.annotations if re.search('BAD', a['description'])]
print(bad_channels)
for s in bad_segments: print('%.2g, %.2g' % (s[0], sum(s)))

# convert to ch mask; and samples to keep/remove
bad_channel_indices = [i for i, ch in enumerate(raw_eo.info['ch_names']) 
                    if ch in bad_channels]
bad_channel_mask = [ch not in bad_channels for ch in channel_names]

[range(int(b * sampling_freq), int((b+d) * sampling_freq)) for b, d in bad_segments]

if bad_segments:
    bad_segment_samples = np.concatenate([range(int(b * sampling_freq), int((b+d) * sampling_freq)) for b, d in bad_segments])
else:
    bad_segment_samples=[]

print('channels to remove: %s' % bad_channel_indices)
print('channel mask: ' + repr(bad_channel_mask))
print('number of samples to remove: %s' % len(bad_segment_samples))

# remove the bad channels and bad samples, make new data matrices:
np_eo_forica = raw_eo.copy().get_data().T

np_eo_forica = np.delete(np_eo_forica, bad_segment_samples, axis=0)
np_eo_forica = np.delete(np_eo_forica, bad_channel_indices, axis=1)

print('raw original shape: %d, %d' % raw_eo.get_data().shape)
print('np new shape: %d, %d' % np_eo_forica.shape)

# # inspect the new data matrix in the viewer
# raw_eo_forica = mne.io.RawArray(np_eo_forica.T,
#                         mne.create_info([n for n in channel_names if n not in bad_channels], 
#                                     sampling_freq, 
#                                     'eeg', 
#                                     None)
#                        )

# raw_eo_forica.plot(scalings='auto');

from pynfb.protocols.ssd.topomap_selector_ica import ICADialog

ica_rejection, _, _, ica_unmixing_matrix, _, _ = ICADialog.get_rejection(
    np_eo_forica, 
    [n for n in channel_names if n not in bad_channels], 
    sampling_freq,
    decomposition=None
)

ica_rejection = ica_rejection.expand_by_mask(bad_channel_mask)

print('Created an ICA Spatial Filter')
print(ica_rejection)

[]
channels to remove: []
channel mask: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
number of samples to remove: 0
raw original shape: 16, 941
np new shape: 941, 16
apply filter: 3 to 45
Dropped 141 outliers
ICA/CSP time elapsed = 0.8720769882202148s
Table drawing time elapsed = 4.004317283630371s


AttributeError: 'NoneType' object has no attribute 'expand_by_mask'

### Start up the Stimulus on Computer
- connect via WiFi
    - ip address = 10.42.0.1
    - host = stim-pc
    - password = PASSWORD, OR 12345678

In [12]:
from callpyff import bcinetwork, bcixml
bcinet = bcinetwork.BciNetwork('10.42.0.1', bcinetwork.FC_PORT, bcinetwork.GUI_PORT, 'bcixml')

In [19]:
print(bcinet.getAvailableFeedbacks())

['TestD2', 'MovingRhomb', 'EOEC', 'LibetClock', 'BrainWaveTraining_II', 'TobiQLAdapter', 'VisualOddballVE', 'EyetrackerFeedback', 'HexoSpeller', 'P300_Rectangle', 'ERPHex', 'BrainWaveTraining', 'StopVigilanceTask', 'FeedbackCursorArrow', 'TrivialPong', 'CheckerboardVEP', 'HexoSpellerVE', 'BoringClock', 'nback_verbal', 'Lesson01', 'BrainPong', 'CakeSpellerVE', 'MovingRhombGL', 'RestingState', 'NFBasicThermometer', 'RSVPSpeller', 'EEGfMRILocalizer', 'MultiVisualOddball', 'Lesson01b', 'GoalKeeper', 'CenterSpellerVE', 'Oddball', 'EyetrackerRawdata', 'StroopFeedback', 'ERPMatrix', 'Lesson04', 'Lesson05', 'Lesson06', 'VisualOddball', 'Lesson02', 'Lesson03']


In [20]:
bcinet.send_init('BrainWaveTraining_II')

In [21]:
bcinet.send_signal(bcixml.BciSignal({'EX_TESTNFNOISE': False},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_PIXWIDTH': 1366},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_PIXHEIGHT': 768},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_FULLSCR': True},None, bcixml.INTERACTION_SIGNAL))

In [22]:
from nftools.nftools import signaltracking

In [23]:
import importlib

In [24]:
importlib.reload(signaltracking)

<module 'nftools.nftools.signaltracking' from '/home/johan/nf/nf-rtime/nftools/nftools/signaltracking.py'>

In [13]:
# parameters to convert the filtered EEG signal to the stimulus
from nftools.nftools import signaltracking
track_for_eeg_stimuli = signaltracking.sending_to_nfstim(
    sampling_freq,
    thr=1.0, 
    dur=0.20, 
    feedback_type='eeg', 
    max4audio=1.2, 
    bcinet=bcinet, 
    st_scaling=10,
    verbose=False
)

# parameters to convert the filtered EMG signal to the stimulus
track_for_emg_stimuli = signaltracking.sending_to_nfstim(
    sampling_freq,
    thr=10, 
    dur=0.15, 
    feedback_type='emg', 
    bcinet=bcinet, 
    st_scaling=30,
    verbose=False
)

thr: 1.00, dur: 0.20
bcinet is passed on
thr: 10.00, dur: 0.15
bcinet is passed on


### Tell the NF Laptop to start!
 - you still have to press <ENTER> to actually start the stimulus, but after starting the NF loop later on

In [4]:
bcinet.play()

NameError: name 'bcinet' is not defined

### Run the MarkerServer in the background
 - this will listen on UDP port 6500 for any incoming markers
 - This is to convert signals from the Presentation into annotations
 - grab markers in the NF loop with: `while not marker_queue.empty():`

In [14]:
from multiprocessing import Process, Queue, Event
import time
if not 'marker_queue' in locals():
    marker_queue = Queue()
if not 'stop_server' in locals():
    stop_server = Event()
from libmushu.ampdecorator import marker_reader

stop_server.set()
time.sleep(0.5)
stop_server.clear()

tcp_reader = Process(target=marker_reader, args=(
    marker_queue,
    Event().set(),
    Event(),
    stop_server,
    6500
    )
)

tcp_reader.start()
# stop the server with: 
# stop_server.set()
# tcp_reader.join()

starting markerserver on port: 6500


### Define Real-Time EEG Signal Analysis during NF

In [15]:
from pynfb.signal_processing.filters import (FilterSequence, 
                                             CFIRBandEnvelopeDetector, 
                                             ExponentialSmoother,
                                             SpatialFilter,
                                             ButterFilter,
                                            )

In [16]:
# Define Filter Sequence for NF

# Which channel do we select for the EEG?
rt_eeg_channels = ['C3']
rt_eeg_channels_mask = np.where([ch in rt_eeg_channels for ch in channel_names], 1, 0)/len(rt_eeg_channels)

preprocess_filters_eeg = FilterSequence([
    ica_rejection,
    SpatialFilter(rt_eeg_channels_mask),
    ButterFilter([1, None], sampling_freq, 1),
])
envelope_filter_eeg = CFIRBandEnvelopeDetector([12, 15], sampling_freq, ExponentialSmoother(0.9))


# which channel for the EMG (Muscle)?
rt_emg_channels = ['T3', 'T4'] # check if they're not in the bad channels..
rt_emg_channels_mask = np.where([ch in rt_eeg_channels for ch in channel_names], 1, 0)/len(rt_eeg_channels)

preprocess_filters_emg = FilterSequence([
    SpatialFilter(rt_emg_channels_mask),
    ButterFilter([None, 50], sampling_freq, 1),
])
envelope_filter_emg = CFIRBandEnvelopeDetector([50, 60], sampling_freq, ExponentialSmoother(0.9))

### Run the experiment
 - grab the data, apply RT Analysis, send to stimulus computer
 - press Enter on the Stimulus Computer to get started
 - subject trans (or should train) ther EEG

In [17]:
# create our UI 'Experience' -- it can consist of 3 separate, movable windows (for now)
# same window as before + 2 other windows - 1 for interaction with stim/thresholds; 1 for looking
# at the analysis itself.
w_acquire = guis.AcquireData(sampling_freq, channel_names)

In [ ]:
w_interaction = guis.NFChangeThresholds(track_for_eeg_stimuli, track_for_eeg_stimuli)

In [1]:
w_eeganalysis = guis.CustomSignalViewer(sampling_freq, ['EEG','env','thr','visual','audio'], overlap=True, notch_filter=False)

NameError: name 'guis' is not defined

In [6]:
w_eeganalysis.show()

In [24]:
w_eeganalysis.show()

In [ ]:
# clear EEG Data buffer
while data_inlet.samples_available(): data_inlet.pull_chunk() 

# containers for data collection: 
time_nf = []
data_nf = []
data_analysis = []


# collect markers (for annotations)
stim_Annotations = mne.Annotations(0, 0, 'Start NF Stim')
# the GUI ones will be in w_analysis.GUI_Annotations



while w_acquire.RUNLOOP:
    
    # check of the presentation gave any markers, and collect them:
    while not marker_queue.empty():
        stim_time, stim_code = marker_queue.get()
        stim_Annotations.append(stim_time, 0, stim_code)
    
    # process the data:
    chunk_data, chunk_times = data_inlet.pull_chunk() # grab from LSL
    if chunk_data:  # if there is any data

        time_nf.append(chunk_times)
        data_nf.append(chunk_data) # add to our list
        
        w_acquire.update(chunk_data) # update the GUI window
        
        
        # check if anything comes from our bcinet; convert those to Annotations
        # ... # for now; just a list - begin and end annotations are separate
        # later we covert to timespans
        # I don't really know whether bcinet can actually read from the stimulus??
        # likely bcinet CAN also send, but exactly how to do this -- pudb on a running
        # feedback instance.
        #
        # alternatively -- start up an LSL and send from fb and connect / receive from over here.
        # I don't think this kind of use-case was readily though of by the pyff folks.
        # but, I think we can try it out.
        # we would need the PUDB in 'remote' mode to access and try this out.
        # all my hard works in python bears fruit...
        #
        # in any case, here we should just check a queue - and if there's contents - we grow our 
        # annotations array.
        
        
        # Apply Filters and draw in our second window

        preprocessed_eeg = preprocess_filters_eeg.apply(chunk_data)
        envelope_eeg = envelope_filter_eeg.apply(preprocessed_eeg)

        preprocessed_emg = preprocess_filters_emg.apply(chunk_data)
        envelope_emg = envelope_filter_eeg.apply(preprocessed_emg)
        
        
        
        # interaction with the Stimulus Ciomputer -- this goes via the GUI's callbacks
        
        
        visual_markers_eeg, audio_markers_eeg = track_for_eeg_stimuli.check_above_threshold(envelope_eeg)  # sends markers (should be fast)
        track_for_eeg_stimuli.send_data_signal(envelope_eeg) # sends the signal (should also be fast!)
        # query the threshold
        # query the duration
        
        visual_markers_emg, audio_markers_emg = track_for_emg_stimuli.check_above_threshold(envelope_emg)  # sends markers (should be fast)
        track_for_emg_stimuli.send_data_signal(envelope_emg) # sends the signal (should also be fast!)

        # we should also pass on the track_for_eeg_stimuli and track_for_emg_stimuli to the GUI, as well
        # as the function of the buttons that they press (lambda functions)
        # so we know in our gui what is happening
        
        # make a matrix to put into the w_analysis...
        analysis_data = np.vstack((
            preprocessed_eeg,
            envelope_eeg,
            track_for_eeg_stimuli.thr * np.ones(preprocessed_eeg.shape)
            visual_markers_eeg,
            audio_markers_eeg,

        )).T
        
        data_analysis.append(analysis_data)  # store it for later conversion
        w_eeganalysis.update(analysis_data)  # put it onto the screen (but keep our on/off button...)
        
        
        # collect the events that we generate - they are in w_analysis.MNEAnnotations
        

        
        
    # after the loop is done -- we can close the windows (optionally) (or with code)
    # and convert signals + analyzed signals into one MNE struct.
    # best way would be to inspect a short recording and deal with allthe issues
    # se also: w_analysis.GUI_Annotations
        
        

sending signal -- I ! - 6 - 3 - True - -1
sending signal! - 3 False - 1698.5897608752598 -- 0.2
sending signal -- I ! - 4 - 7 - True - -1
sending signal! - 7 False - 3115.7205400830403 -- 0.25
sending signal! - 10 False - 7338.542447950324 -- 0.25
sending signal! - 15 False - 13612.712324363889 -- 0.3
sending signal! - 18 False - 16176.525104721888 -- 0.3
sending signal! - 27 False - 20804.649195549086 -- 0.35
sending signal! - 30 False - 22190.178177309404 -- 0.35
sending signal! - 39 False - 29632.885221581822 -- 0.39999999999999997
sending signal! - 42 False - 33096.352050780304 -- 0.39999999999999997
sending signal! - 50 False - 44230.58759935846 -- 0.44999999999999996
sending signal! - 54 False - 49065.492819553874 -- 0.44999999999999996
sending signal! - 62 False - 62035.14703327439 -- 0.49999999999999994
sending signal! - 64 False - 63504.37080444319 -- 0.49999999999999994
sending signal! - 75 False - 83979.30517311534 -- 0.5499999999999999
sending signal! - 75 False - 86092.178